In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import zipfile
os.chdir("../soheil")

zf = zipfile.ZipFile('sample_submission.zip') 
sample_submission = pd.read_csv(zf.open('sample_submission.csv'))

zf = zipfile.ZipFile('test.zip') 
test = pd.read_csv(zf.open('test.csv'))

zf = zipfile.ZipFile('train.zip') 
train = pd.read_csv(zf.open('train.csv'))


In [3]:
train.head(10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
5,id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443
6,id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341
7,id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551
8,id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255
9,id0012891,2,2016-03-10 21:45:01,2016-03-10 22:05:26,1,-73.981049,40.744339,-73.973000,40.789989,N,1225


In [4]:
#Calculating between pick up and drop off locations.

from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0
 
train['lat'] = np.radians(train.pickup_latitude) - np.radians(train.dropoff_latitude)
train['lon'] = np.radians(train['pickup_longitude']) - np.radians(train['dropoff_longitude'])


train['dist'] = np.sin(train['lat'] / 2)**2 + \
                         np.cos(np.radians(train.pickup_latitude)) * \
                         np.cos(np.radians(train.dropoff_latitude)) * \
                         np.sin(train['lon'] / 2)**2
                         
                         
train['dist2'] = 2 * np.arctan2(np.sqrt(train['dist']), np.sqrt(1 -train['dist'] ))
                         
train['dist_final'] = R * train['dist2']
                         
                         
                         

In [5]:
train.shape

(1458644, 16)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 16 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
lat                   1458644 non-null float64
lon                   1458644 non-null float64
dist                  1458644 non-null float64
dist2                 1458644 non-null float64
dist_final            1458644 non-null float64
dtypes: float64(9), int64(3), object(4)
memory usage: 178.1+ MB


In [7]:
train.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,lat,lon,dist,dist2,dist_final
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02,-1.533458e-05,-1.227978e-06,1.865923e-07,5.400822e-04,3.441944e+00
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03,6.054188e-04,8.126620e-04,9.509836e-06,6.743899e-04,4.297887e+00
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00,-9.345884e-02,-6.359695e-02,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02,-2.442119e-04,-2.432798e-04,9.346112e-09,1.933506e-04,1.232223e+00
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02,-3.595267e-06,-2.343582e-05,2.699982e-08,3.286324e-04,2.094374e+00
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03,2.342250e-04,1.966744e-04,9.250045e-08,6.082777e-04,3.876554e+00
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06,1.942636e-01,1.318237e-01,9.454336e-03,1.947746e-01,1.241298e+03


In [8]:
train.drop(columns = ['lat', 'lon', 'dist', 'dist2'], inplace = True)

In [9]:
test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [10]:
test['lat'] = np.radians(test.pickup_latitude) - np.radians(test.dropoff_latitude)
test['lon'] = np.radians(test['pickup_longitude']) - np.radians(test['dropoff_longitude'])


test['dist'] = np.sin(test['lat'] / 2)**2 + \
                         np.cos(np.radians(test.pickup_latitude)) * \
                         np.cos(np.radians(test.dropoff_latitude)) * \
                         np.sin(test['lon'] / 2)**2
                         
                         
test['dist2'] = 2 * np.arctan2(np.sqrt(test['dist']), np.sqrt(1 -test['dist'] ))
                         
test['dist_final'] = R * test['dist2']
                         
test['dist_final'].head()

0    2.747288
1    2.760105
2    1.306565
3    5.270742
4    0.961143
Name: dist_final, dtype: float64

In [11]:
test.drop(columns = ['lat', 'lon', 'dist', 'dist2'], inplace = True)

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625134 entries, 0 to 625133
Data columns (total 10 columns):
id                    625134 non-null object
vendor_id             625134 non-null int64
pickup_datetime       625134 non-null object
passenger_count       625134 non-null int64
pickup_longitude      625134 non-null float64
pickup_latitude       625134 non-null float64
dropoff_longitude     625134 non-null float64
dropoff_latitude      625134 non-null float64
store_and_fwd_flag    625134 non-null object
dist_final            625134 non-null float64
dtypes: float64(5), int64(2), object(3)
memory usage: 47.7+ MB


In [13]:
test.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,dist_final
count,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000,625134.000000
mean,1.534884,1.661765,-73.973614,40.750927,-73.973458,40.751816,3.434489
std,0.498782,1.311293,0.073389,0.029848,0.072565,0.035824,4.273579
min,1.000000,0.000000,-121.933128,37.389587,-121.933327,36.601322,0.000000
25%,1.000000,1.000000,-73.991852,40.737392,-73.991318,40.736000,1.232316
50%,2.000000,1.000000,-73.981743,40.754093,-73.979774,40.754543,2.094395
75%,2.000000,2.000000,-73.967400,40.768394,-73.963013,40.769852,3.883779
max,2.000000,9.000000,-69.248917,42.814938,-67.496796,48.857597,1036.368707


In [14]:
#Missing values
np.sum(pd.isnull(train))


id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dist_final            0
dtype: int64

In [15]:
np.sum(pd.isnull(test))


id                    0
vendor_id             0
pickup_datetime       0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
dist_final            0
dtype: int64

In [16]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_final
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1.498991
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1.806074
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,6.387103
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1.485965
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1.188962


In [17]:
train['pickup_date'] = pd.to_datetime(train.pickup_datetime)
train['pickup_hour'] =train['pickup_date'].dt.hour
train['pickup_min'] = train['pickup_date'].dt.minute


In [18]:
train['day_name'] = train.pickup_date.dt.day_name()

In [19]:
test['pickup_date'] = pd.to_datetime(test.pickup_datetime)
test['pickup_hour'] =test['pickup_date'].dt.hour
test['pickup_min'] = test['pickup_date'].dt.minute
test['day_name'] = test.pickup_date.dt.day_name()

In [20]:
# Removing redundant features

train.drop(columns =['pickup_datetime', 'dropoff_datetime', 'pickup_date'], inplace =True)
test.drop(columns =['pickup_datetime',  'pickup_date'],                     inplace =True)


In [21]:
train.drop(columns = ['id', 'vendor_id', 'store_and_fwd_flag'], inplace = True)
test.drop(columns = ['id', 'vendor_id', 'store_and_fwd_flag'], inplace = True)


In [22]:
# Check out numbers of duplicated rows
np.sum(train.duplicated())

train.drop_duplicates(inplace = True)

In [23]:
np.sum(test.duplicated())
test.drop_duplicates(inplace = True)

In [24]:
#Change categorical variables to numeric ones
train = pd.get_dummies(data = train, drop_first = True)
test =  pd.get_dummies(data = test, drop_first = True)


In [25]:
#Scale variables
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train.drop(columns = ['trip_duration']))
Y_scaled = np.log1p(train['trip_duration'])

In [26]:
#Model with XGBoost
import xgboost as xgb
from sklearn.metrics import mean_squared_error



In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, Y_scaled, test_size=0.2, random_state=42)

In [28]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 70, verbosity =0)

In [29]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)


C:\Users\soheil\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [30]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.458976


# k-fold Cross Validation using XGBoost
# 


In [31]:
data_dmatrix = xgb.DMatrix(data=X_train_scaled,label=Y_scaled)

params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10,'verbosity'  :0}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=42)



C:\Users\soheil\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [32]:
cv_results.head()


,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,5.428136,0.000491,5.428066,0.001088
1,4.892565,0.003172,4.892617,0.004169
2,4.413204,0.005547,4.413154,0.005989
3,3.980007,0.005890,3.979923,0.005619
4,3.593869,0.006754,3.593762,0.006453
